In [1]:
"""
Name: con_fitting.ipynb
Authors: Christian
Example of fitting simulation to data
"""
%load_ext autoreload
%autoreload 2

In [2]:
 # General imports
import logging
logging.basicConfig(level="WARN")
import cProfile, pstats, io
from pstats import SortKey

import pyabc    
import numpy as np
import matplotlib.pyplot as plt
import sys
import scipy
import os
import tempfile
from tqdm import tqdm
import pandas as pd
from time import time
# Adding path to module
sys.path.append("../")
# picture path
PICS = '../pics/'
# Module imports
from contagion import Contagion, config
import contagion
from contagion.config import ConfigClass, _baseconfig




In [3]:
from contagion.pdfs import Gamma

In [4]:
g = Gamma(2, 1)
vals = g.rvs(10000)
print(np.average(vals))
print(np.std(vals))

2.002314316769109
0.9890481268341429


In [5]:
# Options
config.update(_baseconfig)
config["average social circle"] = 20
config["population size"] = 10000
config["store population"] = True
config["re-use population"] = False
config["infectious duration pdf"] = "gamma"
config["latency duration pdf"] = "gamma"
config["incubation duration pdf"] = "gamma"
config["hospitalization duration pdf"] = "gamma"
config["time incubation death pdf"] = "gamma"
config["recovery time pdf"] = "gamma"
config["time until hospitalization pdf"] = "gamma"


In [30]:
data_np = np.load("../data/cities/munich_march.npy")

data_sane = np.zeros(np.max(data_np[0]))
data_sane[data_np[0]-1] = data_np[1]

data = {"is_infected": data_sane}

In [6]:
config["simulation length"] = 50

#profiler = cProfile.Profile()

contagion = Contagion()
#profiler.enable()
contagion.sim()
#profiler.disable()
#profiler.dump_stats("profile.prof")

In [7]:
fields = ["is_infected", "is_dead"]
data = {field: np.asarray(contagion.statistics[field]) for field in fields}

In [10]:
def model(parameters):
    for key, val in parameters.items():
        key = key.replace("_", " ")
        config[key] = val
    contagion = Contagion()
    contagion.sim()
    return contagion.statistics

def make_distance_func(fields):
    dfuncs = []
    for field in fields:
        def distance(simulation, data):
            return np.absolute(data[field] - simulation[field]).sum()
        dfuncs.append(distance)
    return pyabc.AggregatedDistance(dfuncs)
distance = make_distance_func(fields)
prior = pyabc.Distribution(infectious_duration_mean=pyabc.RV("uniform", 1, 10))

In [11]:
config['debug level']

30

In [13]:
logging.getLogger().setLevel("INFO")
config["re-use population"] = True
abc = pyabc.ABCSMC(model, prior, distance,  population_size=100,)
#db_path = "sqlite:///" + os.path.join(tempfile.gettempdir(), "tmp.db")
db_path = "sqlite:///" + os.path.join("C:/Users/Christian/Documents/repos/contagion/examples/", "abc.db")
abc.new(db_path, data)
history1 = abc.run(max_nr_populations=10)
#history1 = abc.load(db_path, 3)

INFO:History:Start <ABCSMC(id=14, start_time=2020-04-02 20:12:18.606908, end_time=None)>
INFO:ABC:Calibration sample before t=0.
INFO:Epsilon:initial epsilon is 106.0
INFO:ABC:t: 0, eps: 106.0.
INFO:ABC:Acceptance rate: 100 / 205 = 4.8780e-01, ESS=1.0000e+02.
INFO:ABC:t: 1, eps: 82.0.
INFO:ABC:Acceptance rate: 100 / 385 = 2.5974e-01, ESS=9.0280e+01.
INFO:ABC:t: 2, eps: 68.90062779193114.
INFO:ABC:Acceptance rate: 100 / 865 = 1.1561e-01, ESS=9.8113e+01.
INFO:ABC:t: 3, eps: 62.0.
INFO:ABC:Acceptance rate: 100 / 1289 = 7.7580e-02, ESS=9.6029e+01.
INFO:ABC:t: 4, eps: 54.0.


KeyboardInterrupt: 

In [17]:
#abc_continued.load(db, run_id)

NameError: name 'abc_continued' is not defined

In [22]:
history1.get_population(1).to_dict()

{0: [<pyabc.population.Particle at 0x1be5a0cadf0>,
  <pyabc.population.Particle at 0x1be5ad69700>]}

In [23]:
history1.get_distribution(m=0, t=t)

(name  infected
 id            
 38        33.0
 39        27.0
 40        26.0
 41        23.0
 42        52.0
 43        12.0
 44        18.0
 45        31.0
 46        25.0
 47        38.0,
 array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]))

In [ ]:
fig, ax = plt.subplots()
for t in range(history1.max_t+1):
    df, w = history1.get_distribution(m=0, t=t)
    pyabc.visualization.plot_kde_1d(
        df, w,
        xmin=0, xmax=20,
        x="infectious_duration_mean", ax=ax,
        label="PDF t={}".format(t))
#ax.axvline(data["is_infected"], color="k", linestyle="dashed");
ax.legend()